In [26]:
import os
import re  # 正则表达式提取文本
from jsonpath import jsonpath  # 解析json数据
import requests  # 发送请求
import pandas as pd  # 存取csv文件
import datetime  # 转换时间用

import time
import re
import random

class Crawler:
    def __init__(self, id = 4860690891018271):
        self.id = id
        self.mid = id
        
        self.cookie = 'WEIBOCN_FROM=1110006030; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFfYb7mkA6BHfpdMmuxGQ8k5JpX5K-hUgL.FoqpShBfe05cSoM2dJLoIE-LxKqLBoMLBo2LxKnL1KeL1-BLxKBLBo.L12zLxK-LB.qL1hSo; SCF=At3cKMF8zNNlQvzopN0jE4GXnW6d1oMHWhTaIC30tCfqPajGN6tNkHrnFHqQBP4OlYtShy_Or1r2ELmJDriqh-I.; SUB=_2A25OzHZUDeRhGeBP71YU8y7KzTuIHXVqTxocrDV6PUJbktAKLVfBkW1NRUYLcaFm2Y6WDEzl3u_svMh_CqKt1m1s; SSOLoginState=1674053124; ALF=1676645124; _T_WM=54103595383; MLOGIN=1; XSRF-TOKEN=f69380; mweibo_short_token=b0e76dc5fb; M_WEIBOCN_PARAMS=oid%3D4860690891018271%26luicode%3D20000061%26lfid%3D4860690891018271%26uicode%3D20000061%26fid%3D4860690891018271'

        self.url = f'https://m.weibo.cn/statuses/extend?id={self.id}'

        r = requests.get(self.url, timeout = 5)
        string = r.json()['data']['longTextContent']
        df = pd.DataFrame(columns = ["index", "text"])
        df = df.append({"index": id, "text": string}, ignore_index = True)
        df.to_csv("spider.csv", index = False)

    def sub_comment(self, comment_id):
        headers = {
        f'Cookie': '{self.cookie}',
        f'Referer': 'https://m.weibo.cn/detail/{self.id}?cid={comment_id}',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.61',
        'X-Requested-With': 'XMLHttpRequest'
        }

        times, max = 0, 1
        max_id, id_type = 0, 0
        temp_index, temp_text = [], []

        while times < max:

            url = f'https://m.weibo.cn/comments/hotFlowChild?cid={comment_id}&max_id={max_id}&max_id_type={id_type}'
            print(url)
            r = requests.get(url, headers = headers, timeout = 5)

            try:
                cards = r.json()['data']
                
                max_id = r.json()['max_id']
                id_type = r.json()['max_id_type']
                if times == 0: max = r.json()['max']

                for i in cards:
                    print(i['text'])
                    temp_index.append(i["id"])
                    temp_text.append(i["text"])

                time.sleep(random.uniform(0, 1))
                times += 1

            except Exception as e:
                print("max not found in sub comment")
                print(e)
                return temp_index, temp_text
        
        return temp_index, temp_text

    def comment(self):
        # 第一条评论
        headers = {
            f'Cookie': '{cookie}',
            'Referer': 'https://m.weibo.cn/detail/4860690891018271',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.61',
            'X-Requested-With': 'XMLHttpRequest'
        }

        times, max = 0, 1
        max_id, id_type = 0, 0

        # 滚动评论
        url = f'https://m.weibo.cn/comments/hotflow?id={self.id}&mid={self.mid}&max_id={max_id}&max_id_type={id_type}' if max_id else f'https://m.weibo.cn/comments/hotflow?id={self.id}&mid={self.mid}&max_id_type=0'
        print(url)

        while times < max:
            r = requests.get(url, headers = headers, timeout = 5)
            cards = r.json()['data']['data']
            temp_index, temp_text = [], []

            time.sleep(random.uniform(0, 2))
            times += 1 

            try:
                for i in cards:
                    print(i['id'], i['text'])
                    temp_index.append(i['id'])
                    temp_text.append(i['text'])

                    a, b = sub_comment(i['id'])
                    temp_index += a
                    temp_text.extend(b)

                    temp_index += ' '
                    temp_text += ' '

                df = pd.DataFrame({"index": temp_index, "text": temp_text})
                # df = df.append({"index": cards['id'], "text": cards['text']}, ignore_index = True)
                
                df.to_csv("spider.csv", index = False, mode = 'a', header = False)

                if times == 0: max = r.json()['max']

                max_id = r.json()['data']['max_id']
                id_type = r.json()['data']['max_id_type']

            except Exception as e:
                print("max not found")
                print(e)
                continue
    
    def trans_time(v_str):
        """转换GMT时间为标准格式"""
        GMT_FORMAT = '%a %b %d %H:%M:%S +0800 %Y'
        timeArray = datetime.datetime.strptime(v_str, GMT_FORMAT)
        ret_time = timeArray.strftime("%Y-%m-%d %H:%M:%S")
        return ret_time

    # pattern = "[\u4e00-\u9fa5]{5}"

    # matches = re.findall(pattern, string)

    # if matches:
    #     for match in matches:
    #         print(match, end = "")
    # else:
    #     print("匹配失败")

	# except:
	# 		continue
	
	# finally:
	# 	time.sleep(random.uniform(0, 2))
	# 	times += 1

In [27]:
if __name__ == '__main__':
    Crawler(4860690891018271).comment()

/tmp/ipykernel_2210310/3412914135.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"index": id, "text": string}, ignore_index = True)


In [ ]:
# # 转发数
# reposts_count_list = jsonpath(cards, '$..mblog.reposts_count')
# # 评论数
# comments_count_list = jsonpath(cards, '$..mblog.comments_count')
# # 点赞数
# attitudes_count_list = jsonpath(cards, '$..mblog.attitudes_count')

# # 请求地址
# url = 'https://m.weibo.cn/api/container/getIndex'
# # 请求参数
# params = {
# 	"containerid": "100103type=1&q={}".format(topic),
# 	"page_type": "searchall",
# 	# "page": searchall
# }
# r = requests.get(url, headers = headers, params = params)

In [ ]:
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

class Crawler:

    def __init__(self, urls=[]):
        self.visited_urls = []
        self.urls_to_visit = urls

    def download_url(self, url):
        return requests.get(url).text

    def get_linked_urls(self, url, html):
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('a'):
            path = link.get('href')
            if path and path.startswith('/'):
                path = urljoin(url, path)
            yield path

    def add_url_to_visit(self, url):
        if url not in self.visited_urls and url not in self.urls_to_visit:
            self.urls_to_visit.append(url)

    def crawl(self, url):
        html = self.download_url(url)
        for url in self.get_linked_urls(url, html):
            self.add_url_to_visit(url)

    def run(self):
        while self.urls_to_visit:
            url = self.urls_to_visit.pop(0)
            logging.info(f'Crawling: {url}')
            try:
                self.crawl(url)
            except Exception:
                logging.exception(f'Failed to crawl: {url}')
            finally:
                self.visited_urls.append(url)

if __name__ == '__main__':
    Crawler(urls=['https://www.imdb.com/']).run()